In [1]:
def get_game(soup):        
 
    #Game Summary
    game_result = soup.find_all("div", class_="match-header")[0]
    home = game_result.find_all("a", class_="team-link")[0]
    away = game_result.find_all("a", class_="team-link")[-1]
    home_team = home.text
    away_team = away.text
    home_id = home['href'][home['href'].find("/",2)+1:home['href'].find("/",home['href'].find("/",2)+1)]
    away_id = away['href'][away['href'].find("/",2)+1:away['href'].find("/",away['href'].find("/",2)+1)]
    half_time = game_result.find_all("dd")[1].text
    home_score_half = half_time[0:half_time.find(':')].replace(' ','')
    away_score_half = half_time[half_time.find(':')+1:len(half_time)].replace(' ','')
    final_time = game_result.find_all("dd")[2].text
    home_score_final = final_time[0:final_time.find(':')].replace(' ','')
    away_score_final = final_time[final_time.find(':')+1:len(final_time)].replace(' ','')
    time1 = game_result.find_all("dd")[3].text
    date = game_result.find_all("dd")[4].text

    col_games = ['GameID','Competition','Season','HomeTeam','HomeTeam_ID','AwayTeam','AwayTeam_ID','HalfTime',
                 'Home_Score_Half','Away_Score_Half','FinalTime','Home_Score_Final','Away_Score_Final',
                 'Time','Date']

    df_games = pd.DataFrame([[game_id,competition,season,home_team, home_id, away_team, away_id, 
                              half_time,home_score_half,away_score_half,final_time,home_score_final,
                              away_score_final,time1,date]],columns=col_games)

    #Game events
    df_events = []
    teams = soup.find_all("tbody", id="player-table-statistics-body")

    events_col=['GameID','ID_Team','Team','Team_Type','ID_Player','Minute','Second','Event_ID']
    df_events = pd.DataFrame(columns=events_col)

    i=1
    for team in teams:            
        for player in team:                

            events = player.find_all("span", class_="incident-icon")  

            for event in events:              
                detail = event.attrs     
                if 'data-type' in detail:
                    if i==1: 
                        team = 'Home' 
                        team_name = home_team
                    elif i==2: 
                        team = 'Away'
                        team_name = away_team
                    player_id_v2 = detail['data-player-id']
                    event_type = detail['data-type'] 
                    minute = detail['data-minute']
                    second = detail['data-second']
                    team_id = detail['data-team-id']     

                    df_events = df_events.append(pd.DataFrame([[game_id,team_id,team_name,team,player_id_v2,minute,
                                                                second,event_type]],columns=events_col))                
        i += 1
        if i == 3: break



    #Game stats    
    type_stat = ['live-player-home-offensive','live-player-home-defensive','live-player-home-passing',
                 'live-player-away-offensive','live-player-away-defensive','live-player-away-passing']
    
    for click in type_stat:
        try:
            driver.find_element_by_css_selector("[href*='"+click+"']").click()
            time.sleep(5)
        except:
            pass
    
    raw_data = driver.page_source
    soup = BeautifulSoup(raw_data, "lxml")

    teams = soup.find_all("tbody", id="player-table-statistics-body")

    df_stats = [None]*8
    col_stats = [None]*4
    col_stats[0] = ['GameID','N','Team_ID','Team','Type','Player_ID','Player_Name','Position','ShotsTotal',
                    'ShotOnTarget','KeyPassTotal','PassSuccessInMatch','DuelAerialWon','Touches','rating']
    col_stats[1] = ['Player_ID','TackleWonTotal','InterceptionAll','ClearanceTotal','ShotBlocked','FoulCommitted']
    col_stats[2] = ['Player_ID','DribbleWon','FoulGiven','OffsideGiven',
                    'Turnover','Dispossessed',]
    col_stats[3] = ['Player_ID','TotalPasses','PassCrossTotal','PassCrossAccurate',
                    'PassLongBallTotal','PassLongBallAccurate','PassThroughBallTotal','PassThroughBallAccurate']

    i=0
    k=0
    df_away = []
    df_home = []

    for team in teams: 
        df_aux = pd.DataFrame(columns=col_stats[k])
        j = 1       
        for player in team: 
            summary = dict()
            plink = player.find_all("a", class_="player-link")[0]["href"]  
            summary['Player_ID'] = plink[9:plink.find("/",9)]        
            if i==0:
                summary['GameID'] = game_id
                summary['Team'] = home_team
                summary['Team_ID'] = home_id
                summary['Type'] = 'Home'
                summary['N'] = j            
                summary['Player_Name'] = plink[plink.find("/Show/")+6:len(plink)]
            elif i==4:
                summary['GameID'] = game_id
                summary['Team'] = away_team
                summary['Team_ID'] = away_id
                summary['Type'] = 'Away'
                summary['N'] = j            
                summary['Player_Name'] = plink[plink.find("/Show/")+6:len(plink)]
            for stat in player:            
                if type(stat) is not bs4.element.NavigableString:
                    if type(stat.get('class')) is list: 
                        if (stat.get('class')[0] == 'col12-lg-2' and (i==0 or i==4)):
                            summary['Position'] = stat.text[stat.text.find(',')+1:min(len(stat.text),stat.text.find('('))].strip()
                        else:
                            summary[stat.get('class')[0]] = stat.text.replace('\t','')             
            df_aux = df_aux.append(pd.DataFrame([summary], columns=col_stats[k]))             
            j += 1

        df_stats[i] = df_aux.copy()
        i += 1    
        k += 1
        if k==4: 
            k=0  

    df_home = pd.merge(df_stats[0],df_stats[1], on='Player_ID')
    df_home = pd.merge(df_home,df_stats[2], on='Player_ID')
    df_home = pd.merge(df_home,df_stats[3], on='Player_ID')
    df_away = pd.merge(df_stats[4],df_stats[5], on='Player_ID')
    df_away = pd.merge(df_away,df_stats[6], on='Player_ID')
    df_away = pd.merge(df_away,df_stats[7], on='Player_ID')
    df_game = pd.concat([df_home, df_away])



    #return results   
    return df_game, df_games, df_events
        

In [2]:
def load_game(game_id):    
    
    link = "https://www.whoscored.com/Matches/" + str(game_id) + "/LiveStatistics"    
    
    try:
        driver.get(link)
        time.sleep(5)
    except:
        print('Timeout error')
        time.sleep(5)
        load_game(game_id)
        return
    
    raw_data = driver.page_source
    soup = BeautifulSoup(raw_data,"lxml") 
    
    if soup.find(attrs={'name': 'ROBOTS'}):  
        print("Access Denied: Error code 15")
        time.sleep(60)
        print("Continue")
        load_game(game_id)
        return
    
    df1=df2=df3=[]
    try:        
        df1,df2,df3 = get_game(soup)          
    except:
        time.sleep(5)
        print('Error processing')
        load_game(game_id)
        return
    
    if (len(df1) >= 22 and len(df2) == 1 and len(df3) > 0):
        df1.to_sql(con=db_conn, name='game_stats', if_exists='append', index=False)
        df2.to_sql(con=db_conn, name='games', if_exists='append', index=False)    
        df3.to_sql(con=db_conn, name='game_events', if_exists='append', index=False)        
    else:
        print('Empty dataframe', len(df1), len(df2), len(df3))
        time.sleep(5)
        load_game(game_id)

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import sqlalchemy
import time
from datetime import datetime

user = 'root'
password = 'flores87'
ip = '127.0.0.1'
db = 'whoscored'
db_conn = sqlalchemy.create_engine('mysql+mysqlconnector://{}:{}@{}/{}'.format(user,password,ip,db))


driver = webdriver.Firefox(executable_path="geckodriver.exe")
driver.set_page_load_timeout(100)

In [4]:
df_games = pd.read_csv('usa_div1_2020_missing.csv')

for index, game in df_games.iterrows():  
    
    competition = game['league']
    division = game['division']
    season = game['season']
    game_id = game['id']

    load_game(game_id)     

In [5]:
'''
game_id
driver = webdriver.Firefox(executable_path="geckodriver.exe")
driver.set_page_load_timeout(100)
link = "https://www.whoscored.com/Matches/" + str(game_id) + "/LiveStatistics"    

driver.get(link)
time.sleep(5)
raw_data = driver.page_source
soup = BeautifulSoup(raw_data,"lxml") 
df1,df2,df3 = get_game(soup)         
df1.to_sql(con=db_conn, name='game_stats', if_exists='append', index=False)
df2.to_sql(con=db_conn, name='games', if_exists='append', index=False)    
df3.to_sql(con=db_conn, name='game_events', if_exists='append', index=False)  

'''

'\ngame_id\ndriver = webdriver.Firefox(executable_path="geckodriver.exe")\ndriver.set_page_load_timeout(100)\nlink = "https://www.whoscored.com/Matches/" + str(game_id) + "/LiveStatistics"    \n\ndriver.get(link)\ntime.sleep(5)\nraw_data = driver.page_source\nsoup = BeautifulSoup(raw_data,"lxml") \ndf1,df2,df3 = get_game(soup)         \ndf1.to_sql(con=db_conn, name=\'game_stats\', if_exists=\'append\', index=False)\ndf2.to_sql(con=db_conn, name=\'games\', if_exists=\'append\', index=False)    \ndf3.to_sql(con=db_conn, name=\'game_events\', if_exists=\'append\', index=False)  \n\n'